In [ ]:
!pip install --extra-index-url https://test.pypi.org/simple/ aecdata

In [ ]:
import os
import pandas as pd 
import numpy as np
from itertools import product
from aecdata import *

### Setup developer token

In [ ]:
# Set the environment variable
os.environ['DEVELOPER_TOKEN']= ''

# Now you can access it using getenv
developer_token = os.getenv('DEVELOPER_TOKEN')

### Create user

In [ ]:
user = User(developer_token = developer_token)

### user attributes

In [ ]:
# user.base_api_url # default = "https://app.2050-materials.com/"
# user.authenticator
# user.api_token
# user.refresh_token

# Lazy calculation of filters and field_description
# user.filters # first time the get_filters method is called and cached as user property
# user.field_description # first time the get_filters method is called and cached as user property

### Refresh token 

In [ ]:
user.refresh_api_token()

### user class has various methods to retrieve utility data

In [ ]:
# user.get_filters_template() 
# field_description = user.get_field_description()
# user.get_input_lca_fields_description()
# user.get_output_lca_fields_description()
# user.get_impact_lca_fields_description()
# user.get_material_facts_fields_description()
# user.get_physical_properties_fields_description()
# user.get_technical_parameters_fields_description()
# user.get_product_fields_description()
# user.get_unit_categories() 
# user.get_primary_units()
# user.get_mf_num_fields()
# user.get_mf_perc_fields()
# user.get_physical_properties_fields()

### Get products

In [ ]:
# products = user.get_products_page(page=2)
# products

### Filters

In [ ]:
# Fetch all filters
all_filters = user.filters

# # Fetch all filter mappings
all_mappings = user.get_filters_mapping()
# all_mappings['material_types_family']

# # Fetch filter template
# filter_template = user.get_filters_template()
# filter_template

# # Fetch filters for open API
# open_filters = user.get_open_filters()


### Apply Filters

In [ ]:
filters = {
#     'name': '12.5 mm Gyproc Fireline',
#     'product_type': [7,8,9,10],
#    'material_types': [all_mappings['material_types']['Ceramic']],
    'material_type_family': all_mappings['material_types_family']['Ceramic'],
#     'country': 'United Kingdom',
#     'manufacturing_continent': 'Europe',
#     'fire_performance': ['  B-s1,d0','M3 (NF P92-501 : 1995)'],
#     'norm_price': 4,
#     'mf_unit': 'kg'
}
filtered_products = user.get_products(**filters) # Add openapi=True for the free version
# filtered_products = user.get_products(page=1, **filters)

## Create ProductData object 

In [ ]:
# Create ProductData object from API data
product_data = ProductData(filtered_products)
df = product_data.dataframe

# Save as csv
# product_data.to_csv('example.csv')

# from Dataframe
product_data_head = ProductData(df.head())
product_data_head.data

### A change in the .data attribute is automatically reflected in the .dataframe attribute (and vice versa)

In [ ]:
# Construct ProductData obj
product_data_head = ProductData(df.head())

# Assign the data attribute of the obj to new_data variable
new_data = product_data_head.data

# Print name of the first product
print('Old data name:', new_data[0]['name'])

# Modify the new_data variable 
new_data[0]['name'] = 'NEW NAME TEST'

# Assign the value to data
product_data_head.data = new_data

# Print the new value from the dataframe
print('New dataframe name:', product_data_head.dataframe.loc[0,'name'])

## Project tool

#### Define custom project using UUID, unit and amount of product

In [ ]:
products_info = {
    '6aa6d32c-f8cf-11ed-9c01-0242ac120004' : {'amount': 0.2}, #{'unit':'declared_unit', 'amount': 0.2},
    'c0800dc0-f8cc-11ed-9c01-0242ac120004' : {'amount': 0.30}, #{'unit':'declared_unit', 'amount': 0.3},
    '6ab16fb2-f8cf-11ed-9c01-0242ac120004' : {'unit':'m2'}, #{'unit':'m2', 'amount': 1},
    '6aab7152-f8cf-11ed-9c01-0242ac120004' : {'unit':'m2', 'amount': 1},
}


#### Create scaled dataframe or plot product contribution to a specified LCA field

In [ ]:
scale_df = product_data.scale_products_by_unit_and_amount(products_info)

product_data.plot_product_contributions(products_info, 'material_facts.manufacturing')

## Convert to EPDx format

In [ ]:
epdx_products = product_data.to_epdx()
# epdx_df = pd.json_normalize(epdx_products)
# epdx_df

## Create ProductStatistics (extension of ProductData)

In [ ]:
stats_obj = ProductStatistics(product_data.dataframe, unit='m2')

#### Calculate statistics

In [ ]:
group_by = [
    # 'group_elements_nrm_1',
    'country',
    'manufacturing_continent',
    # 'material_facts.data_source',
]
all_available_fields  = stats_obj.get_available_fields()
stat_df = stats_obj.get_statistics(group_by=group_by, fields=all_available_fields, statistical_metrics=['count', 'mean', 'median'], include_estimated_values=False, remove_outliers=True, method='IQR', sqrt_tranf=True, min_count=3)
stat_df

#### Another example

In [ ]:
group_by = [
    # 'company',
    # 'product_type',
    'material_type', 
    # 'material_type_family',
    # 'elements_nrm_1',
    # 'country',
    # 'manufacturing_continent',
    # 'material_facts.data_source',
]
all_available_fields  = stats_obj.get_available_fields()
stat_df = stats_obj.get_statistics(group_by=group_by, fields='all', statistical_metrics=['count', 'mean', 'median'], include_estimated_values=False, remove_outliers=True, method='IQR', sqrt_tranf=True, min_count=3)
stat_df

### Field distribution

#### Without removing outliers  

In [ ]:
filters = {
    'material_type':'Ceramic',
    # 'elements_nrm_1':'3.1 - Wall finishes'
}
stats_obj.get_field_distribution(field='material_facts.manufacturing', filters=filters, include_estimated_values=True, remove_outliers=False, method='IQR', sqrt_tranf=True)

#### With removing outliers

In [ ]:
stats_obj.get_field_distribution(field='material_facts.manufacturing', filters=filters, include_estimated_values=True, remove_outliers=True, method='IQR', sqrt_tranf=True)

### Field distribution boxplot

In [ ]:
stats_obj.get_field_distribution_boxplot(field='material_facts.manufacturing', group_by_field='product_type', filters=None, include_estimated_values=True, remove_outliers=True, method='IQR', sqrt_tranf=True)
